# Vector/Pipelining scenario #1: RGB to YUV conversion

Example RGB to YUV conversion (simplified, not clamping)

**Note** Not verified for correct conversion, legacy example taken from MyHDL implementation. Can be implemented more elegantly.

Makes use of the `VectorSig` data type and the simple `@pipeline` decorator.

In [1]:
!pip install numpy > /dev/null

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


Import video types:

In [2]:
from video.color import *
from video.videotypes import *

Import pipeline and target auxiliaries:

In [3]:
from myirl.library.pipeline import *
from myirl import targets

Construct the conversion matrix, in this case for JPEG-compliant YCrCb:

In [4]:
CLAMP = False
LEVELSHIFT = False
BPP        = 8
FRACT_SIZE = 16
CALCSIZE   = FRACT_SIZE + BPP
SATURATION_VALUE_MAX = 127 # YUV maximum value (saturation)
SATURATION_VALUE_MIN = -128 # YUV minimum value (saturation)


# Signed matrix entries:
Y_FROM_RGB = vector_to_fp(FRACT_SIZE, 1, mat_jpeg_rgb2yuv[0])
U_FROM_RGB = vector_to_fp(FRACT_SIZE, 1, mat_jpeg_rgb2yuv[1])
V_FROM_RGB = vector_to_fp(FRACT_SIZE, 1, mat_jpeg_rgb2yuv[2])

def F(x, s = FRACT_SIZE):
    return intbv(x)[s:]

YUV_SLICE = slice(CALCSIZE-1, CALCSIZE-1 - BPP)

MATRIX = [
    [ F(Y_FROM_RGB[i]) for i in range(3) ],
    [ F(U_FROM_RGB[i]) for i in range(3) ],
    [ F(V_FROM_RGB[i]) for i in range(3) ]
]

from myirl.library.vectorsignal import VectorSignal

I = lambda x: ( x[i]._val for i in range(3) )

# @bulkwrapper()
# class RGBParam:
#     def __init__(self):
#         self.y = VectorSig(3, MATRIX[0], initializer = I(MATRIX[0]))
#         self.u = VectorSig(3, MATRIX[1], initializer = I(MATRIX[1]))
#         self.v = VectorSig(3, MATRIX[1], initializer = I(MATRIX[2]))

In [5]:
MATRIX

[[intbv(9797), intbv(19234), intbv(3735)],
 [intbv(60007), intbv(54682), intbv(16384)],
 [intbv(16384), intbv(51817), intbv(62872)]]

In [6]:
from myirl import simulation as sim
from myirl.test.common_test import gen_osc

@block
def video_rgb_yuv(clk : ClkSignal,
                  vin : VideoPort,
                  rgb : Signal,
                  param_matrix : list,
                  vout : VideoPort.Output,
                  yuv : Signal.Output,):
    """RGB to full range YUV422 converter, manual pipeline inference"""
    
    py, pu, pv = [
        VectorSignal(3, F(0), initializer = I(param_matrix[i]), name = "p_coef%d" % i)  \
        for i in range(3)
    ]
    
    # Use initializers:
    py._init = True
    pu._init = True
    pv._init = True
    
    valid = Signal(bool())
    
    rgb_v = VectorSignal(3, FractUnsigned(0, BPP), name = 'rgbv')
    
    a = VectorSignal(3, FractSigned(0, CALCSIZE+2), name = "add_res")
    y = VectorSignal(3, FractUnsigned(0, CALCSIZE), name = "ydata")
    u, v = [ VectorSignal(3, FractSigned(0, CALCSIZE+1), name = n) for n in ['udata', 'vdata'] ]

    # Wire up input RGB video:
    wires = []
    for i in range(3):
        j = 3 - i
        wires.append(rgb_v[i].wireup(rgb[j*BPP:(j-1)*BPP]))

    # Predefine YUV slices
    yuv_slices = (a[i][YUV_SLICE] for i in range(3) )
         
    wires += [
        yuv.wireup(
            concat(*yuv_slices)
        )   
    ]
    
    @pipeline(clk, None, ce = vin.dval, pass_in = vin, pass_out = vout)
    def yuv_pipe(ctx):
        """This contains the two-stage transformation for the RGB-YUV matrix.
Because it's a vector signal, we can use HDL notation (<=)"""
        yield [        
            y .set (py * rgb_v),
            u .set (pu.signed() * rgb_v),
            v .set (pv.signed() * rgb_v)        
        ]

        # Create sum expressions for readability:
        _y, _u, _v = (i.sum() for i in [y, u, v])
        
        yield [ 
            a[0].set(_y.signed()),
            a[1].set(_u),
            a[2].set(_v) 
        ]

    return locals()


### Testbench

In [7]:
from myirl.targets import VHDL
from myirl.test.common_test import run_ghdl

d = DesignModule("top", debug = True)

@component(d)
def testbench_rgb2yuv():
    clk = ClkSignal(name = "pclk")
    yuv = Signal(intbv(0)[3*BPP:])
    vint, vout = [VideoPort() for _ in range(2)]
    
    yuv = Signal(intbv(0)[3*BPP:], name = 'yuv_data')
    rgb = Signal(intbv(0)[3*BPP:], name = 'rgb_data')

    
    inst = video_rgb_yuv(clk = clk,
                        vin = vint,
                        rgb = rgb,
                        param_matrix = MATRIX,
                        vout = vout,
                        yuv = yuv
    )
    
    osc = gen_osc(clk, CYCLE = 5)
    
    @sim.generator
    def stimulus():    
        
        # Feed a few color values:
        values = sim.Iterator([0x00ffff, 0x7f7f7f, 0x008300, 0x1a840a])
        
        yield [
            vint.dval.set(False), vint.fval.set(True), vint.lval.set(True),
            sim.wait(4 * [ clk.posedge, ] ), 
            vint.dval.set(True),
            sim.For(values)(
                sim.wait('1 ns'),
                rgb.set(values),
                sim.wait(2 * [clk.posedge]),
                sim.print_(yuv),
            ),

            sim.wait(3 * [ clk.posedge, ] ), 
            sim.assert_(vout.dval == True, "Video not valid"),
        ]

        for _ in range(3):
            yield [
                sim.print_(yuv),
                sim.wait(clk.posedge), 
            ]

        yield [
            sim.raise_(sim.StopSimulation)
        ]
        
    return locals()

def test():
    tb = testbench_rgb2yuv()
    files = tb.elab(VHDL, elab_all = True)
    run_ghdl(files, tb, debug = True, vcdfile="yuv.vcd")
    return files, tb

 Declare obj 'testbench_rgb2yuv' in context '(DesignModule 'top')'(<class 'myirl.kernel.instance.DesignModule'>) 


In [8]:
files, tb = test()

/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/components.py:116: UserWarning: Fallback: Pass through other argument for arg param_matrix (<class 'list'>)
  base.warnings.warn(msg)
/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/library/pipeline.py:285: TranslationWarning: yuv_pipe(): `ce` (type <class 'myirl.kernel._types.ChildAlias'>) is not a pipeline signal
  base.warn("%s(): `ce` (type %s) is not a pipeline signal" % (func.__name__, type(ce)))


 Insert unit video_rgb_yuvu_1dval_1u_lval_1u_fval_1u_24_l3dval_1u_lval_1u_fval_1u_24 
 Insert unit testbench_rgb2yuv 
 DEBUG REGISTER module_defs: VideoPort 
 DEBUG REGISTER top: VideoPort 
 DEBUG REGISTER module_defs: VideoPort 
 DEBUG REGISTER top: VideoPort 
 DEBUG REGISTER module_defs: VideoPort 
 DEBUG REGISTER top: VideoPort 
 DEBUG REGISTER module_defs: VideoPort 
 DEBUG REGISTER top: VideoPort 
 DEBUG REGISTER module_defs: VideoPort 
 DEBUG REGISTER top: VideoPort 
 DEBUG REGISTER module_defs: VideoPort 
 Writing 'video_rgb_yuv' to file /tmp/video_rgb_yuv.vhdl 
 Writing 'testbench_rgb2yuv' to file /tmp/testbench_rgb2yuv.vhdl 
 Creating library file module_defs.vhdl 
WORK DIR of instance [Instance testbench_rgb2yuv I/F: [// ID: testbench_rgb2yuv_0 ]] /tmp/
==== COSIM stdout ====
0xuuuuuu
0xB22B80
0x7E0000
0x4CD4C9
0x56D4D4
0x56D4D4
0x56D4D4
simulation stopped @175ns



In [9]:
!cat -n {files[0]}

     1	-- File generated from source:
     2	--     /tmp/ipykernel_97023/3889866912.py
     3	-- (c) 2016-2022 section5.ch
     4	-- Modifications may be lost, edit the source file instead.
     5	
     6	library IEEE;
     7	use IEEE.std_logic_1164.all;
     8	use IEEE.numeric_std.all;
     9	
    10	library work;
    11	
    12	use work.module_defs.all;
    13	use work.txt_util.all;
    14	use work.myirl_conversion.all;
    15	
    16	
    17	
    18	entity video_rgb_yuv is
    19	    port (
    20	        clk : in std_ulogic;
    21	        vin : in t_VideoPort;
    22	        rgb : in unsigned(23 downto 0);
    23	        vout : out t_VideoPort;
    24	        yuv : out unsigned(23 downto 0)
    25	    );
    26	end entity video_rgb_yuv;
    27	
    28	architecture myIRL of video_rgb_yuv is
    29	    -- Local type declarations
    30	    -- Signal declarations
    31	    signal yuv_pipe_ce1 : std_ulogic;
    32	    signal yuv_pipe_ce2 : std_ulogic;
    33	    signal yuv_pipe_ce0 :

## Waveform trace

In [10]:
from cyrite import waveutils

waveutils.draw_wavetrace(tb, 'yuv.vcd', 'pclk', cfg = None)

Download full VCD trace [yuv.vcd](yuv.vcd)

## Verification exercise

Using numpy, we can run our samples through the floating point matrix as well:

In [11]:
v = numpy.matrix(mat_jpeg_rgb2yuv)
rgb = numpy.matrix([ (127, 127, 127), (0, 255, 255), (0, 0x83, 0)]).T

yuv = v * rgb

In [12]:
g = lambda x: "%02x" % (int(x) & 0xff)
f = numpy.vectorize(g)
f(yuv.T)

matrix([['7e', '00', '00'],
        ['b2', '2b', '81'],
        ['4c', 'd5', 'ca']], dtype='<U2')

We note that the results don't entirely match. Why?